## Presentation Type Classifier (with Hidden Types)

#### Also Network Visualization with Louvain Communities--See below

* May 2022 version

    * Uses getDistance to identify `close matches` with side-by-side comparison of soggetti.  With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same.  These are reported as "flexed entries" in a separate column.

    * Labels Fuga, PEn, and ID according to time intervals.  
    * If two entries are separated by **more than 10 bars (80 offsets)**, the tool resets to a new pattern
    * Finds **time intervals between entries* (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds **melodic intervals between first note of successive entries** in each pattern (like `P-5, P-8`)
    * Counts **number of entries**
    * Provides **offset and measure/beat locations**
    * Sorts all presentation types by the **order in which they appear in the piece**
    * Reports **voice names of the entries, in order of their appearance**
    * **Omits singleton soggetti** (just one entry of a given motive in isolation)
    
* Options
    
    * set the **length of the soggetti** with `melodic_ngram_length` (4 is a good default)
    * set the **maximum difference between similar soggetti** with `edit_distance_threshold` (0 will yield only identical soggetti; 1 will allow one total integer difference across all the intervals)
    * for **chromatic vs diatonic, compound, and zero-based diatonic in soggetti**, use interval_settings (`kind= 'd', or 'c', or 'z'`)
    * to include all the **hidden PENs and IDS** (those found within longer Fugas, use `include_hidden_types == True`. That is, if a 5-voice fuga also contains a PEN, it will label both of these as separate presentation type, along with all the relevant data noted above. For faster (and simpler) listing of points of imitation **without hidden forms**, use `include_hidden_types == False`
    * **combineUnisons** via the initial import of the NotesRests.  Select this with `True` or `False`.  Combining Unisons can be a good way to identify connections across compositions
    * **Limit_to_Entries** allows you to choose whether the P Type finder considers **only soggetti that enter after a rest or section break** or instead considers **all ngrams** as a moving window in each voice.  The latter will likely produce large numbers of overlapping Presentation Types of the same basic type and time/melodic entries.  

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


#### The following are special functions used by the classifier.  Don't change them.

## Load one Piece Here

* Note that you can load from CRIM, or put a file in the **Music_Files** folder in the Notebook.

In [2]:
prefix = 'https://crimproject.org/mei/'
# just add the CRIM Piece ID here
mei_file = 'CRIM_Model_0050.mei'
url = prefix + mei_file

piece = importScore(url)


print(piece.metadata)

Successfully imported https://crimproject.org/mei/CRIM_Model_0050.mei
{'title': 'Domine Dominus noster', 'composer': 'Roland de  Lassus'}


## Find Presentation Types

* `piece.presentationTypes()`



- set the length of the soggetti with `melodic_ngram_length`
- set the maximum difference between similar soggetti with `edit_distance_threshold`
- for chromatic vs diatonic, compound, and directed data in soggetti, see `interval_settings`
- to include all the hidden PENs and IDS (those found within longer Fugas, use `include_hidden_types == True`.  
- for faster (and simpler) listing of points of imitation without hidden forms, use `include_hidden_types == False`



In [3]:
piece.presentationTypes(edit_distance_threshold = 0,
                        include_hidden_types = False,
                        combine_unisons = True,
                       melodic_ngram_length = 4)


,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,Roland de Lassus,Domine Dominus noster,0.0,"[1/1.0, 7/2.0]",[P4],"[0.0, 50.0]","[(-2, 2, -3, 2)]",[50.0],"[Altus, Sexta]",FUGA,2,False


In [5]:
output

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,"Lupi, Johannes",Veni speciosam,0.0,"[1/1.0, 2/3.0, 5/1.0, 6/3.0, 9/2.0, 10/1.0]","[P-4, P-5, P-4, P4, P8]","[0.0, 12.0, 32.0, 44.0, 66.0, 72.0]","[(5, -2, 2, 3), (4, -2, 2, 3)]","[12.0, 20.0, 12.0, 22.0, 6.0]","[Superius, Contratenor, PrimusTenor, Bassus, S...",FUGA,6,True
1,"Lupi, Johannes",Veni speciosam,94.0,"[12/4.0, 16/4.0]",[P4],"[94.0, 126.0]","[(-3, 2, 2, -2)]",[32.0],"[Bassus, Bassus]",FUGA,2,False
2,"Lupi, Johannes",Veni speciosam,138.0,"[18/2.0, 18/4.0, 19/4.0, 20/4.0, 21/2.0, 24/2....","[P-4, P4, P5, P5, M-9, P-4, P4]","[138.0, 142.0, 150.0, 158.0, 162.0, 186.0, 190...","[(-2, -3, 2, 2)]","[4.0, 8.0, 8.0, 4.0, 24.0, 4.0, 8.0]","[PrimusTenor, Bassus, SecundusTenor, Contraten...",FUGA,8,False
3,"Lupi, Johannes",Veni speciosam,226.0,"[29/2.0, 30/3.0, 31/2.0, 31/3.0, 34/4.0, 35/2....","[P-4, P8, P-4, P4, P5, P-8]","[226.0, 236.0, 242.0, 244.0, 270.0, 274.0, 290.0]","[(5, -2, 2, 3), (4, -2, 2, 3)]","[10.0, 6.0, 2.0, 26.0, 4.0, 16.0]","[PrimusTenor, Bassus, Superius, SecundusTenor,...",FUGA,7,True
4,"Lupi, Johannes",Veni speciosam,314.0,"[40/2.0, 41/2.0, 41/3.0, 42/2.0, 43/2.0, 43/4....","[P8, P-5, P-4, P-5, m9, m-6, P12, P-8]","[314.0, 322.0, 324.0, 330.0, 338.0, 342.0, 350...","[(2, -3, 2, 3), (2, -3, 2, 4)]","[8.0, 2.0, 6.0, 8.0, 4.0, 8.0, 4.0, 8.0]","[Bassus, Superius, PrimusTenor, SecundusTenor,...",FUGA,9,True
5,"Lupi, Johannes",Veni speciosam,382.0,"[48/4.0, 49/1.0]",[P-4],"[382.0, 384.0]","[(-2, -2, -2, 5), (-2, -2, -2, 4)]",[2.0],"[PrimusTenor, Bassus]",FUGA,2,True
6,"Lupi, Johannes",Veni speciosam,394.0,"[50/2.0, 51/4.0, 53/2.0, 54/3.0]","[P5, P1, P-8]","[394.0, 406.0, 418.0, 428.0]","[(3, 2, -3, -2)]","[12.0, 12.0, 10.0]","[SecundusTenor, PrimusTenor, SecundusTenor, Ba...",FUGA,4,False
7,"Lupi, Johannes",Veni speciosam,430.0,"[54/4.0, 55/2.0, 56/2.0, 58/2.0, 58/4.0, 60/4....","[P4, P5, M-9, P5, P-4, P-5, P4, P1, P-5, P5]","[430.0, 434.0, 442.0, 458.0, 462.0, 478.0, 490...","[(4, 1, -2, 2), (5, 1, -2, 2), (5, 1, -3, 2)]","[4.0, 8.0, 16.0, 4.0, 16.0, 12.0, 2.0, 14.0, 8...","[PrimusTenor, Contratenor, Superius, Bassus, P...",FUGA,11,True
8,"Lupi, Johannes",Veni speciosam,430.0,"[54/4.0, 55/2.0, 56/2.0, 56/4.0, 58/2.0, 58/4....","[P4, P5, M-9, P1, P5, P-4, P-5, P4, P-5, P5]","[430.0, 434.0, 442.0, 446.0, 458.0, 462.0, 478...","[(4, 1, -2, 2), (4, 1, -2, 1), (5, 1, -2, 2)]","[4.0, 8.0, 4.0, 12.0, 4.0, 16.0, 12.0, 2.0, 22...","[PrimusTenor, Contratenor, Superius, SecundusT...",FUGA,11,True
9,"Lupi, Johannes",Veni speciosam,430.0,"[54/4.0, 55/2.0, 56/2.0, 56/4.0, 58/4.0, 60/4....","[P4, P5, M-9, P5, P-4, P-5, P4, P1]","[430.0, 434.0, 442.0, 446.0, 462.0, 478.0, 490...","[(4, 1, -2, 2), (4, 1, -2, 1)]","[4.0, 8.0, 4.0, 16.0, 16.0, 12.0, 2.0, 26.0]","[PrimusTenor, Contratenor, Superius, SecundusT...",FUGA,9,True


In [4]:
output.to_csv("Model_0001.csv")

## Find Presentation Types in Several Pieces at Once

Results are combined into a single dataframe

In [4]:
prefix = 'https://crimproject.org/mei/'

piece_list = ['CRIM_Model_0019',
             'CRIM_Mass_0019_1',
             'CRIM_Mass_0019_2',
             'CRIM_Mass_0019_3',
             'CRIM_Mass_0019_4',
             'CRIM_Mass_0019_5']

suffix = '.mei'

In [5]:

final = pd.DataFrame()

for piece in piece_list:
    file_url = prefix + piece 
    url = prefix + piece + suffix
    piece = importScore(url)   
    output = piece.presentationTypes(edit_distance_threshold = 0,
                        include_hidden_types = False,
                        combine_unisons = True,
                       melodic_ngram_length = 4)

    final['Validation'] = ""
    final['Comments'] = ""
    final = final.append(output, ignore_index=True)
final.head()
# final.to_csv("corpus_classified.csv")
# final.to_csv("saved_csv/CRIM_Mass_0019_P_Types_hidden.csv")

Successfully imported https://crimproject.org/mei/CRIM_Model_0019.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_1.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_2.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_3.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_4.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_5.mei


,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8]","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0]","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,7.0,False
1,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,150.0,"[19/4.0, 21/4.0, 22/4.0, 25/4.0, 27/4.0, 29/4....","[P8, P-5, P-4, P-5, P8, P8, P-11]","[150.0, 166.0, 174.0, 198.0, 214.0, 230.0, 250...","[(-3, 2, 2, -2)]","[16.0, 8.0, 24.0, 16.0, 16.0, 20.0, 24.0]","[Tenor, Cantus, Altus, Tenor, Bassus, Altus, C...",FUGA,8.0,False
2,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 42/2.0, 44/1.0, 46/2.0, 48/4.0]","[P-5, P-5, P8, P8, P-11]","[298.0, 302.0, 330.0, 344.0, 362.0, 382.0]","[(-3, 3, -2, -2)]","[4.0, 28.0, 14.0, 18.0, 20.0]","[Altus, Tenor, Bassus, Altus, Cantus, Bassus]",FUGA,6.0,False
3,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4.0, 59/1....","[P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[394.0, 406.0, 418.0, 434.0, 446.0, 464.0, 486...","[(-2, 2, 3, -2)]","[12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0, 16.0]","[Altus, Cantus, Bassus, Tenor, Altus, Bassus, ...",FUGA,9.0,False
4,,,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4.0,False


In [6]:
final

""


# Network Visualization with Louvain Communities

* This will only make sense for SETS of pieces!

### Add Combined Data for Melodic, Time and Soggetti as "Words" for Network Edges

In [6]:
final["MINT"] = final["Melodic_Entry_Intervals"].apply(joiner)
final["TINT"] = final["Time_Entry_Intervals"].apply(joiner)

final['SOG'] = final['Soggetti'].apply(clean_melody_new)
final['ALL'] = final["MINT"] + '_' + final["TINT"] + '_' + final['SOG']

final["ALL_INT"] = final["MINT"] + '_' + final["TINT"]
final["MINT_SOG"] = final["MINT"] + '_' + final["SOG"]
final["TINT_SOG"] = final["TINT"] + '_' + final["SOG"]
final

,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,...,Presentation_Type,Number_Entries,Flexed_Entries,MINT,TINT,SOG,ALL,ALL_INT,MINT_SOG,TINT_SOG
0,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8]","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0]",...,FUGA,7.0,False,P-5_P-4_P-5_P8_P5_P-8,8.0_32.0_8.0_36.0_20.0_24.0,-3_3_2_-2,P-5_P-4_P-5_P8_P5_P-8_8.0_32.0_8.0_36.0_20.0_2...,P-5_P-4_P-5_P8_P5_P-8_8.0_32.0_8.0_36.0_20.0_24.0,P-5_P-4_P-5_P8_P5_P-8_-3_3_2_-2,8.0_32.0_8.0_36.0_20.0_24.0_-3_3_2_-2
1,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,150.0,"[19/4.0, 21/4.0, 22/4.0, 25/4.0, 27/4.0, 29/4....","[P8, P-5, P-4, P-5, P8, P8, P-11]","[150.0, 166.0, 174.0, 198.0, 214.0, 230.0, 250...","[(-3, 2, 2, -2)]","[16.0, 8.0, 24.0, 16.0, 16.0, 20.0, 24.0]",...,FUGA,8.0,False,P8_P-5_P-4_P-5_P8_P8_P-11,16.0_8.0_24.0_16.0_16.0_20.0_24.0,-3_2_2_-2,P8_P-5_P-4_P-5_P8_P8_P-11_16.0_8.0_24.0_16.0_1...,P8_P-5_P-4_P-5_P8_P8_P-11_16.0_8.0_24.0_16.0_1...,P8_P-5_P-4_P-5_P8_P8_P-11_-3_2_2_-2,16.0_8.0_24.0_16.0_16.0_20.0_24.0_-3_2_2_-2
2,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 42/2.0, 44/1.0, 46/2.0, 48/4.0]","[P-5, P-5, P8, P8, P-11]","[298.0, 302.0, 330.0, 344.0, 362.0, 382.0]","[(-3, 3, -2, -2)]","[4.0, 28.0, 14.0, 18.0, 20.0]",...,FUGA,6.0,False,P-5_P-5_P8_P8_P-11,4.0_28.0_14.0_18.0_20.0,-3_3_-2_-2,P-5_P-5_P8_P8_P-11_4.0_28.0_14.0_18.0_20.0_-3_...,P-5_P-5_P8_P8_P-11_4.0_28.0_14.0_18.0_20.0,P-5_P-5_P8_P8_P-11_-3_3_-2_-2,4.0_28.0_14.0_18.0_20.0_-3_3_-2_-2
3,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4.0, 59/1....","[P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[394.0, 406.0, 418.0, 434.0, 446.0, 464.0, 486...","[(-2, 2, 3, -2)]","[12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0, 16.0]",...,FUGA,9.0,False,P5_P-8_P4_P5_P-8_P8_P-4_P-5,12.0_12.0_16.0_12.0_18.0_22.0_4.0_16.0,-2_2_3_-2,P5_P-8_P4_P5_P-8_P8_P-4_P-5_12.0_12.0_16.0_12....,P5_P-8_P4_P5_P-8_P8_P-4_P-5_12.0_12.0_16.0_12....,P5_P-8_P4_P5_P-8_P8_P-4_P-5_-2_2_3_-2,12.0_12.0_16.0_12.0_18.0_22.0_4.0_16.0_-2_2_3_-2
4,,,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0]",...,ID,4.0,False,P-5_P-4_P-5,8.0_32.0_8.0,-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0_-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0,P-5_P-4_P-5_-3_3_2_-2,8.0_32.0_8.0_-3_3_2_-2
5,,,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,78.0,"[10/4.0, 12/2.0, 14/2.0, 15/3.0]","[P-4, P-5, P-4]","[78.0, 90.0, 106.0, 116.0]","[(2, 2, 2, 2)]","[12.0, 16.0, 10.0]",...,FUGA,4.0,False,P-4_P-5_P-4,12.0_16.0_10.0,2_2_2_2,P-4_P-5_P-4_12.0_16.0_10.0_2_2_2_2,P-4_P-5_P-4_12.0_16.0_10.0,P-4_P-5_P-4_2_2_2_2,12.0_16.0_10.0_2_2_2_2
6,,,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,160.0,"[20/1.0, 24/1.0, 30/2.0, 33/1.0, 35/3.0, 39/3.0]","[P-8, P1, P4, P-8, P5]","[160.0, 192.0, 242.0, 264.0, 284.0, 316.0]","[(-3, 2, 2, -2)]","[32.0, 50.0, 22.0, 20.0, 32.0]",...,FUGA,6.0,False,P-8_P1_P4_P-8_P5,32.0_50.0_22.0_20.0_32.0,-3_2_2_-2,P-8_P1_P4_P-8_P5_32.0_50.0_22.0_20.0_32.0_-3_2...,P-8_P1_P4_P-8_P5_32.0_50.0_22.0_20.0_32.0,P-8_P1_P4_P-8_P5_-3_2_2_-2,32.0_50.0_22.0_20.0_32.0_-3_2_2_-2
7,,,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,164.0,"[20/3.0, 24/3.0, 27/2.0, 27/4.0, 30/4.0, 36/1....","[P-8, P8, P-8, P5, P4, P5, M-6]","[164.0, 196.0, 218.0, 222.0, 246.0, 288.0, 322...","[(2, 2, -2, -3)]","[32.0, 22.0, 4.0, 24.0, 42.0, 34.0, 4.0]",...,FUGA,8.0,False,P-8_P8_P-8_P5_P4_P5_M-6,32.0_22.0_4.0_24.0_42.0_34.0_4.0,2_2_-2_-3,P-8_P8_P-8_P5_P4_P5_M-6_32.0_22.0_4.0_24.0_42....,P-8_P8_P-8_P5_P4_P5_M-6_32.0_22.0_4.0_24.0_42....,P-8_P8_P-8_P5_P4_P5_M-6_2_2_-2_-3,32.0_22.0_4.0_24.0_42.0_34.0_4.0_2_2_-2_-3
8,,,Giovanni

### Make Groups, Based on Selected Criteria:

* MINT
* TINT
* SOG
* Or any of the combinations:
    * ALL (MINT + TINT + Soggetti)
    * ALL_INT (MINT + TINT)
    * MINT_SOG (MINT + Soggetti)
    * TINT_SOG (TINT + Soggetti)

In [7]:

output_grouped = final.groupby('SOG')['Title'].apply(list).reset_index()
output_grouped


,SOG,Title
0,-2_-2_-2_-2,"[Missa Veni sponsa Christi: Credo, Missa Veni ..."
1,-2_-2_-2_2,[Missa Veni sponsa Christi: Credo]
2,-2_-2_3_2,[Missa Veni sponsa Christi: Credo]
3,-2_-3_3_-2,[Missa Veni sponsa Christi: Gloria]
4,-2_2_-2_-2,[Missa Veni sponsa Christi: Credo]
5,-2_2_-3_2,"[Missa Veni sponsa Christi: Credo, Missa Veni ..."
6,-2_2_2_-2,[Missa Veni sponsa Christi: Agnus Dei]
7,-2_2_2_2,"[Missa Veni sponsa Christi: Kyrie, Missa Veni ..."
8,-2_2_3_-2,"[Veni sponsa Christi, Missa Veni sponsa Christ..."
9,-3_2_2_-2,"[Veni sponsa Christi, Missa Veni sponsa Christ..."


In [8]:
pairs = output_grouped.Title.apply(lambda x: list(combinations(x, 2)))
pairs

0     [(Missa Veni sponsa Christi: Credo, Missa Veni...
1                                                    []
2                                                    []
3                                                    []
4                                                    []
5     [(Missa Veni sponsa Christi: Credo, Missa Veni...
6                                                    []
7     [(Missa Veni sponsa Christi: Kyrie, Missa Veni...
8     [(Veni sponsa Christi, Missa Veni sponsa Chris...
9     [(Veni sponsa Christi, Missa Veni sponsa Chris...
10                                                   []
11                                                   []
12    [(Veni sponsa Christi, Missa Veni sponsa Chris...
13    [(Veni sponsa Christi, Missa Veni sponsa Chris...
14                                                   []
15                                                   []
16                                                   []
17                                              

In [9]:
pairs2 = pairs.explode().dropna()
pairs2

0     (Missa Veni sponsa Christi: Credo, Missa Veni ...
5     (Missa Veni sponsa Christi: Credo, Missa Veni ...
7     (Missa Veni sponsa Christi: Kyrie, Missa Veni ...
8     (Veni sponsa Christi, Missa Veni sponsa Christ...
8     (Veni sponsa Christi, Missa Veni sponsa Christ...
                            ...                        
13    (Missa Veni sponsa Christi: Sanctus, Missa Ven...
13    (Missa Veni sponsa Christi: Sanctus, Missa Ven...
13    (Missa Veni sponsa Christi: Sanctus, Missa Ven...
13    (Missa Veni sponsa Christi: Agnus Dei, Missa V...
18    (Missa Veni sponsa Christi: Kyrie, Missa Veni ...
Name: Title, Length: 132, dtype: object

In [10]:
unique_pairs = pairs.explode().dropna().unique()
unique_pairs

array([('Missa Veni sponsa Christi: Credo', 'Missa Veni sponsa Christi: Sanctus'),
       ('Missa Veni sponsa Christi: Kyrie', 'Missa Veni sponsa Christi: Sanctus'),
       ('Veni sponsa Christi', 'Missa Veni sponsa Christi: Kyrie'),
       ('Veni sponsa Christi', 'Missa Veni sponsa Christi: Gloria'),
       ('Veni sponsa Christi', 'Missa Veni sponsa Christi: Credo'),
       ('Veni sponsa Christi', 'Missa Veni sponsa Christi: Sanctus'),
       ('Veni sponsa Christi', 'Missa Veni sponsa Christi: Agnus Dei'),
       ('Missa Veni sponsa Christi: Kyrie', 'Missa Veni sponsa Christi: Gloria'),
       ('Missa Veni sponsa Christi: Kyrie', 'Missa Veni sponsa Christi: Credo'),
       ('Missa Veni sponsa Christi: Kyrie', 'Missa Veni sponsa Christi: Agnus Dei'),
       ('Missa Veni sponsa Christi: Gloria', 'Missa Veni sponsa Christi: Gloria'),
       ('Missa Veni sponsa Christi: Gloria', 'Missa Veni sponsa Christi: Credo'),
       ('Missa Veni sponsa Christi: Gloria', 'Missa Veni sponsa Christi: S

In [11]:
pd.Series(unique_pairs).isna().sum()

0

In [12]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G

In [13]:
pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="white", font_color="black")

In [14]:
G = nx.Graph()
G.add_edges_from(unique_pairs)
G = add_communities(G)

In [15]:
pyvis_graph.from_nx(G)

In [16]:
pyvis_graph.show('MINT.html')
